In [1]:
import torch
from torch.autograd import Variable
import pandas as pd
import numpy as np

In [2]:
inputSize = 3
outputSize = 1

In [3]:
class LinearRegression(torch.nn.Module):
    def __init__(self,inputSize,outputSize):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize,outputSize)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [4]:
model = LinearRegression(inputSize,outputSize)

In [5]:
model.parameters

<bound method Module.parameters of LinearRegression(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)>

In [6]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.007)

# Let's import data

In [7]:
data = pd.read_csv('./assets/data')

In [8]:
data.head()

,Unnamed: 0,numberOfCars,time,dayNight,Price
0,0,1,6,0,7.11
1,1,1,12,0,7.96
2,2,1,12,1,8.13
3,3,1,6,1,8.79
4,4,2,3,0,7.09


In [9]:
data = data.drop('Unnamed: 0',axis=1)

In [10]:
data.head()

,numberOfCars,time,dayNight,Price
0,1,6,0,7.11
1,1,12,0,7.96
2,1,12,1,8.13
3,1,6,1,8.79
4,2,3,0,7.09


In [11]:
xData = data.drop('Price',axis=1)

In [12]:
xData.head()

,numberOfCars,time,dayNight
0,1,6,0
1,1,12,0
2,1,12,1
3,1,6,1
4,2,3,0


In [13]:
yData = data['Price']

In [14]:
yData.head()

0    7.11
1    7.96
2    8.13
3    8.79
4    7.09
Name: Price, dtype: float64

In [15]:
xData.shape, yData.shape

((80, 3), (80,))

## Data splitting

In [16]:
xDataTrain = xData[:60]
xDataTest = xData[60:]

yDataTrain = yData[:60]
yDataTest = yData[60:]

print(xDataTrain.shape, xDataTest.shape, yDataTrain.shape, yDataTest.shape)

(60, 3) (20, 3) (60,) (20,)


In [17]:
#Convert to Numpy then torch Tensor
xDataTrain = torch.from_numpy(np.array(xDataTrain))
xDataTest = torch.from_numpy(np.array(xDataTest))

yDataTrain = torch.from_numpy(np.array(yDataTrain))
yDataTest = torch.from_numpy(np.array(yDataTest))

In [18]:
xDataTrain.shape, yDataTrain.shape, xDataTest.shape, yDataTest.shape

(torch.Size([60, 3]), torch.Size([60]), torch.Size([20, 3]), torch.Size([20]))

# Let's Train

In [19]:
for epoch in range(2000):
    
    inputs = Variable(xDataTrain)
    inputs = inputs.type(torch.FloatTensor)
    labels = Variable(yDataTrain)
    labels = labels.type(torch.FloatTensor)
    
    optimizer.zero_grad()
    
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    
    loss.backward()
    optimizer.step()
    
    #print("Training Loss",loss.item())
    
    if epoch % 50 == 0:
        
        testInputs = Variable(xDataTest)
        testInputs = testInputs.type(torch.FloatTensor)
        testLabels = Variable(yDataTest)
        testLabels = testLabels.type(torch.FloatTensor)
        
        testOutputs = model(testInputs)
        testLoss = criterion(testOutputs, testLabels)
        
        print('Epoch',epoch,'Loss',testLoss.item())
        

Epoch 0 Loss 290.239990234375
Epoch 50 Loss 13.354681968688965
Epoch 100 Loss 8.952467918395996
Epoch 150 Loss 7.536163806915283
Epoch 200 Loss 6.442272186279297
Epoch 250 Loss 5.543388843536377
Epoch 300 Loss 4.7956414222717285
Epoch 350 Loss 4.167969703674316


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([60])) that is different to the input size (torch.Size([60, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 400 Loss 3.6371216773986816
Epoch 450 Loss 3.1853690147399902
Epoch 500 Loss 2.7989447116851807
Epoch 550 Loss 2.4670519828796387
Epoch 600 Loss 2.181061267852783
Epoch 650 Loss 1.933980941772461
Epoch 700 Loss 1.7201071977615356
Epoch 750 Loss 1.5346965789794922
Epoch 800 Loss 1.3737938404083252
Epoch 850 Loss 1.2340545654296875
Epoch 900 Loss 1.1126494407653809
Epoch 950 Loss 1.0071502923965454
Epoch 1000 Loss 0.9154836535453796
Epoch 1050 Loss 0.8358539342880249
Epoch 1100 Loss 0.7667133212089539
Epoch 1150 Loss 0.7067213654518127
Epoch 1200 Loss 0.6547055244445801
Epoch 1250 Loss 0.6096511483192444
Epoch 1300 Loss 0.5706712603569031
Epoch 1350 Loss 0.5369924306869507
Epoch 1400 Loss 0.5079392790794373
Epoch 1450 Loss 0.48291996121406555
Epoch 1500 Loss 0.4614177644252777
Epoch 1550 Loss 0.442980021238327
Epoch 1600 Loss 0.42721229791641235
Epoch 1650 Loss 0.4137668013572693
Epoch 1700 Loss 0.40234121680259705
Epoch 1750 Loss 0.39266958832740784
Epoch 1800 Loss 0.3845204412937

# Cheers! Now let's save the model 

In [20]:
model.state_dict().keys()

odict_keys(['linear.weight', 'linear.bias'])

In [21]:
torch.save(model.state_dict() , './assets/checkpoint.pth')

## Testing

In [22]:
with torch.no_grad():
    test = torch.tensor([10,3,0])
    test = test.type(torch.FloatTensor)
    prediction = model(test)
print(prediction)

tensor([8.4987])
